<a href="https://colab.research.google.com/github/lucarameh/2nd-Project/blob/main/Image_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hide
! [ -e /content ] & pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 719 kB 31.4 MB/s 
     |████████████████████████████████| 4.9 MB 51.7 MB/s 
     |████████████████████████████████| 432 kB 57.6 MB/s 
     |████████████████████████████████| 1.3 MB 51.5 MB/s 
     |████████████████████████████████| 212 kB 70.8 MB/s 
     |████████████████████████████████| 115 kB 73.2 MB/s 
     |████████████████████████████████| 163 kB 58.1 MB/s 
     |████████████████████████████████| 127 kB 70.7 MB/s 
     |████████████████████████████████| 6.6 MB 54.4 MB/s 
Mounted at /content/gdrive


In [ ]:
#hide
from fastbook import *

In [ ]:
key = os.environ.get('AZURE_SEARCH_KEY', '4e18c3dd7d9340c0af339e0d67bc8da9')

In [ ]:
animals = 'lion','tiger','leopard'
path = Path('animal')

In [ ]:
if not path.exists():
    path.mkdir()
    for o in animals:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, o)
        download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
fns = get_image_files(path)
fns

(#413) [Path('animal/lion/e24f31cc-324a-49c3-b265-8488a6f450e1.gif'),Path('animal/lion/89dd9305-784e-441d-9cf5-8c0e8c56a103.jpg'),Path('animal/lion/9f33d066-43c6-4e0d-adf2-bec5c6cb8b54.jpg'),Path('animal/lion/c2ff8c01-320b-497b-a965-f2294849f497.jpg'),Path('animal/lion/62fb408e-13d2-4e89-bac6-1d1f1b77bf7a.jpg'),Path('animal/lion/5da174aa-b672-4f9f-9242-085081d88d0d.gif'),Path('animal/lion/3ee367c8-ed66-4843-a05a-c55b8ae19b00.jpg'),Path('animal/lion/392f7aa3-33bb-4a82-84ab-a74d4c2e2120.jpg'),Path('animal/lion/4c11385b-cf2c-487e-949d-b60412d6521b.jpg'),Path('animal/lion/b8e76fc4-c265-481d-8db6-85c0125f6c77.jpg')...]

In [ ]:
failed = verify_images(fns)
failed

(#5) [Path('animal/lion/921692dd-97b9-47f8-812c-a6d7fd3c0ada.jpg'),Path('animal/lion/198d2c7e-4211-418b-97e8-a683a6a22b72.jpg'),Path('animal/lion/9dd8dfa5-25ae-44fc-aa0b-c42e18c66845.jpg'),Path('animal/lion/a21e2362-fef5-4f67-a981-5012bb9d840b.jpg'),Path('animal/tiger/b2da9eb6-1d77-4d05-9104-4ae719284d16.jpg')]

In [ ]:
failed.map(Path.unlink);

In [ ]:
animal = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
animal = animal.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = animal.dataloaders(path)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,1.411692,0.486597,0.172840,00:22


epoch,train_loss,valid_loss,error_rate,time
0,0.228187,0.120210,0.037037,00:21
1,0.144532,0.057619,0.012346,00:21
2,0.110929,0.031897,0.012346,00:21
3,0.088902,0.031302,0.012346,00:24


In [ ]:
learn.export()